### REGISTRY
- OpenMMLab은 풍부한 알고리즘 과 데이터 셋 집합을 지원한다. 따라서, 유사 기능을 가진 많은 모듈이 구현되어 있다.
- 예를 들어, ResNet과 SE-ResNet은 각각 '클래스 ResNet'과 'SEResNet'을 기반으로 하고 있으며, 유사한 함수와 인터페이스를 가지며, (OpenMMLab의) 알고리즘 라이브러리의 모델 구성요소에 속하고 있다.
- 해당 모듈들을 관리하게 위해 MM 라이브러리는 registry를 사용한다.

#### 레지스트리란 무엇인가
- 레지스트리는 일종의 매핑테이블이며, 사용자가 입력한 문자열(string)과 대응되는 함수와 클래스를 정의한 것이다.
- 모듈 빌드 함수는 대응되는 클래스 혹은 함수를 어떻게 객체화 하고 호출할 것인지를 정의한다.
- build_from_cfg는 기본적으로 문자열과 대응되는 클래스와 함수를 찾는 함수이다.

- 예를 들어, 레지스트리 MODELS는 모든 모델의 추상화로 간주될 수 있다.  
  ResNet, SEResNet, RegNetX와 같은 클래스를 관리하거나, build_ResNet, build_SEResNet, build_RegNetx과 같은 생성자를 관리한다.

#### Getting started


In [13]:
from mmengine import Registry
# 'scope'는 레지스트리 도메인을 나타내고, 설정되지 않으면 (호출한) 패키지 이름을 가리킨다. e.g. mm3d
# 'location'은 어디 모듈에 호출하는 레지스트리가 정의되어 있는지 나타낸다.
ACTIVATION = Registry('activation', scope='mmengine', locations=['mmengine.models.activations'])

location 인자로 'mmengine.models.activations' 모듈이 구체화 되었고, mmengine/models/activations.py에 대응된다.  
--> 위의 행위가 구현된 모듈을 임포트 하는 행위인가 보다

In [14]:
import torch.nn as nn

# 클래스 이름 Sigmoid가 Registry 클래스의 메소드 register_module의 인자로 들어간다.
# use the register_module 
@ACTIVATION.register_module()
class Sigmoid(nn.Module): 
    def __init__(self):
        super().__init__()

    def forward(self, x):
        print('call Sigmoid.forward')
        return x

@ACTIVATION.register_module()
class ReLU(nn.Module):
    def __init__(self, inplace=False):
        super().__init__()

    def forward(self, x):
        print('call ReLU.forward')
        return x

@ACTIVATION.register_module() # same as @ACTIVATION.register_module(module=Softmax)
class Softmax(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        print('call Softmax.forward')
        return x
    

In [15]:
print(ACTIVATION.module_dict)

{'Sigmoid': <class '__main__.Sigmoid'>, 'ReLU': <class '__main__.ReLU'>, 'Softmax': <class '__main__.Softmax'>}


- registry 모듈 사용의 핵심은 구현된 모듈을 ACTIVATION 레지스트에서 등록하는 것이다.
- 데코레이터를 쓰거나,  다음과 같이 사용해도 된다.

In [16]:
# 이하 Registry 클래스 메소드 일부
    def _register_module(self,
                         module: Type,
                         module_name: Optional[Union[str, List[str]]] = None,
                         force: bool = False) -> None:
        """Register a module.

        Args:
            module (type): Module to be registered. Typically a class or a
                function, but generally all ``Callable`` are acceptable.
            module_name (str or list of str, optional): The module name to be
                registered. If not specified, the class name will be used.
                Defaults to None.
            force (bool): Whether to override an existing class with the same
                name. Defaults to False.
        """
        if not callable(module):
            raise TypeError(f'module must be Callable, but got {type(module)}')

        if module_name is None:
            module_name = module.__name__
        if isinstance(module_name, str):
            module_name = [module_name]
        for name in module_name:
            if not force and name in self._module_dict:
                existed_module = self.module_dict[name]
                raise KeyError(f'{name} is already registered in {self.name} '
                               f'at {existed_module.__module__}')
            self._module_dict[name] = module # 모듈을 Registry 클래스의 멤버인 _module_dict 딕셔너리에 저장한다.

IndentationError: unexpected indent (1198796832.py, line 2)

※ NOTE
- 레지스트르 메커니즘을 활성화 하는 핵심은 모듈이 임포트 되게 하는 것이다.
- 레지스트리에 등록하는 3가지 방법이 있다.
  - 1. 모듈을 location에 정의된 곳에 구현하는 것이다. Registry는 모듈을 자동으로 임포트한다. 사용자는 REGISTRY.build(cfg)로 사용할 수 있다.

Q. 등록하는 것(.register_module())과 빌드(.build) 하는 것의 차이는?
- 등록하는 것은 구현하여 등록할때, 빌드는 사용할 때

In [ ]:
import torch

input = torch.randn(2)

act_cfg = dict(type='Sigmoid')
activation = ACTIVATION.build(act_cfg)
output = activation(input)
print(output) # call Sigmoid.forward

act_cfg = dict(type='ReLU', inplace=True)
activation = ACTIVATION.build(act_cfg)
output = activation(input)
print(output) # call ReLU.forward

- 객체를 생성하기 전에 입력 파라미터의 타입을 체크해 보고자 한다면, build 메로스를 구현하여 레지스트리에 전달하면 된다.

In [ ]:
# Create a build_activation function
def build_activation(cfg, registry, *args, **kwargs):
    cfg_ = cfg.copy()
    act_type = cfg_.pop('type')
    print(f'build activation: {act_type}')

ACTIVATION = Registry('activation', build_func=build_activation, scope='mmengine', locations=['mmengine.models.activations'])

@ACTIVATION.register_module()
class Tanh(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        print('call Tanh.forward')
        return x

act_cfg = dict(type='Tanh')
activation = ACTIVATION.build(act_cfg)
output = activation(input)
# build activation: Tanh
# call Tanh.forward
print(output)

#### [CLASS mmengine.registry.Registry(name, build_func=None, parent=None, scope=None, locations=[])](https://mmengine.readthedocs.io/en/latest/api/generated/mmengine.registry.Registry.html#mmengine.registry.Registry)  
- 문자열을 클래스나 함수에 매핑하는 레지스트리  
- 등록된 객체는 레지스트리에서 빌드 될 수 있다. 등록된 함수는 레지스트리에서 호출될 수 있다.
- 

#### mmengine.registry.build_from_cfg(cfg, registry, default_args=None)
: cfg (dict or ConfigDict or Config) – Config dict. It should at least contain the key “type”.  
: registry (Registry) – The registry to search the type from.  
: default_args (dict or ConfigDict or Config, optional) – Default initialization arguments. Defaults to None.  


In [ ]:
# 사용법 예시
from mmengine import Registry, build_from_cfg
MODELS = Registry('models')

@MODELS.register_module() # ResNet 클래스를 등록한다. 
class ResNet:
     def __init__(self, depth, stages=4):
        self.depth = depth
        self.stages = stages    

cfg = dict(type='ResNet', depth=50)
#model = build_from_cfg(cfg, MODELS)
resnet = MODELS.build(dict(type='ResNet'))

# Returns an instantiated object
@MODELS.register_module()
def resnet50():           # Resnet 함수를 등록한다.
    pass
#reset = build_from_cfg(dict(type='reset50'), MODELS)
# Return a result of the calling function

resnet = MODELS.build(dict(type='resnet50'))